In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import networkx as nx
import scglue
import anndata as ad

/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/envs/multiomics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
cite = sc.read("/mnt/ps/home/CORP/johnny.xi/sandbox/matching/data/GSE194122_openproblems_neurips2021_cite_BMMC_processed.h5ad")

/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/envs/multiomics/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
cite = cite[cite.obs.batch.isin(["s1d1", "s1d2", "s1d3"])]

In [81]:
d = cite.obs.reset_index()

In [83]:
d[:1000]

,index,GEX_n_genes_by_counts,GEX_pct_counts_mt,GEX_size_factors,GEX_phase,ADT_n_antibodies_by_counts,ADT_total_counts,ADT_iso_count,cell_type,batch,...,DonorID,DonorAge,DonorBMI,DonorBloodType,DonorRace,Ethnicity,DonorGender,QCMeds,DonorSmoker,is_train
0,GCATTAGCATAAGCGG-1-s1d1,893,6.723979,0.356535,G1,115,2828.0,5.0,Naive CD20+ B IGKC+,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
1,TACAGGTGTTAGAGTA-1-s1d1,2606,8.008829,1.292643,S,137,8819.0,21.0,CD14+ Mono,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
2,AGGATCTAGGTCTACT-1-s1d1,1867,6.959707,0.970558,G2M,116,4088.0,12.0,Naive CD20+ B IGKC+,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
3,GTAGAAAGTGACACAG-1-s1d1,2360,6.109234,1.232604,G2M,124,4447.0,9.0,HSC,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
4,TCCGAAAAGGATCATA-1-s1d1,455,0.294394,0.044585,S,132,12875.0,24.0,Reticulocyte,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GTGAGCCAGCTGGTGA-1-s1d1,2210,4.479669,1.167016,G2M,117,3423.0,8.0,CD4+ T naive,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
996,GGAATCTTCTATGCCC-1-s1d1,2061,12.652705,1.059667,G2M,115,4047.0,4.0,CD8+ T naive,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
997,AGTCACAAGGATACCG-1-s1d1,1831,5.403879,0.867009,G2M,127,5777.0,8.0,CD4+ T activated,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train
998,GGGAAGTAGTCATGCT-1-s1d1,1884,4.792519,0.977159,G1,117,4259.0,10.0,CD4+ T naive,s1d1,...,15078,34,24.8,B-,White,HISPANIC OR LATINO,Male,False,Nonsmoker,train


In [6]:
adt_ad = cite[:,cite.var.feature_types == "ADT"]

In [7]:
gex_ad = cite[:,cite.var.feature_types == "GEX"]

In [8]:
gex_ad.X = gex_ad.layers["counts"].copy()
sc.pp.normalize_total(gex_ad)
sc.pp.log1p(gex_ad)
sc.pp.scale(gex_ad)
sc.tl.pca(gex_ad, n_comps=100, svd_solver="auto")

/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/envs/multiomics/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [9]:
adt_ad.X = adt_ad.layers["counts"].copy()
sc.pp.normalize_total(adt_ad)
sc.pp.log1p(adt_ad)
sc.pp.scale(adt_ad)
sc.tl.pca(adt_ad, n_comps=100, svd_solver="auto")

/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/envs/multiomics/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [10]:
p = np.array(adt_ad.var_names)
r = np.array(gex_ad.var_names)
# mask entries are set to 1 where protein name is the same as gene name
mask = np.repeat(p.reshape(-1, 1), r.shape[0], axis=1) == r
mask = np.array(mask)

In [11]:
rna_vars = [v + "_rna" for v in gex_ad.var_names]
prot_vars = [v + "_prot" for v in adt_ad.var_names]
gex_ad.var_names = rna_vars
adt_ad.var_names = prot_vars

In [12]:
adj = pd.DataFrame(mask, index=prot_vars, columns=rna_vars)
diag_edges = adj[adj > 0].stack().index.tolist()
diag_edges = [(n1, n2, {"weight": 1.0, "sign": 1}) for n1, n2 in diag_edges]
self_loop_rna = [(g, g, {"weight": 1.0, "sign": 1}) for g in rna_vars]
self_loop_prot = [(g, g, {"weight": 1.0, "sign": 1}) for g in prot_vars]

In [13]:
graph = nx.Graph()
graph.add_nodes_from(rna_vars)
graph.add_nodes_from(prot_vars)
graph.add_edges_from(diag_edges)
graph.add_edges_from(self_loop_prot)
graph.add_edges_from(self_loop_rna)

In [14]:
graph.number_of_nodes(), graph.number_of_edges()

(14087, 14123)

In [15]:
scglue.models.configure_dataset(
    gex_ad,
    "NB",
    use_highly_variable=False,
    use_batch="Samplename",
    use_layer="counts",
    use_rep="X_pca",
)

In [16]:
scglue.models.configure_dataset(
    adt_ad,
    "Normal",
    use_highly_variable=False,
    use_batch="Samplename",
    use_rep="X_pca",
)

In [17]:
glue = scglue.models.fit_SCGLUE(
    {"rna": gex_ad, "adt": adt_ad},
    graph,
)

[INFO] fit_SCGLUE: Pretraining SCGLUE model...
[INFO] autodevice: Using GPU 0 as computation device.
[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!
[INFO] SCGLUEModel: Setting `graph_batch_size` = 4868
[INFO] SCGLUEModel: Setting `max_epochs` = 105
[INFO] SCGLUEModel: Setting `patience` = 9
[INFO] SCGLUEModel: Setting `reduce_lr_patience` = 5
[INFO] SCGLUETrainer: Using training directory: "/tmp/GLUETMPknvocgmm"


/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


[INFO] SCGLUETrainer: [Epoch 10] train={'g_nll': 0.142, 'g_kl': 0.015, 'g_elbo': 0.157, 'x_rna_nll': 0.38, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.384, 'x_adt_nll': 0.915, 'x_adt_kl': 0.109, 'x_adt_elbo': 1.024, 'dsc_loss': 0.611, 'vae_loss': 1.414, 'gen_loss': 1.383}, val={'g_nll': 0.131, 'g_kl': 0.015, 'g_elbo': 0.146, 'x_rna_nll': 0.376, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.379, 'x_adt_nll': 0.887, 'x_adt_kl': 0.11, 'x_adt_elbo': 0.997, 'dsc_loss': 0.603, 'vae_loss': 1.381, 'gen_loss': 1.351}, 5.7s elapsed
[INFO] SCGLUETrainer: [Epoch 20] train={'g_nll': 0.054, 'g_kl': 0.016, 'g_elbo': 0.07, 'x_rna_nll': 0.376, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.379, 'x_adt_nll': 0.879, 'x_adt_kl': 0.124, 'x_adt_elbo': 1.002, 'dsc_loss': 0.641, 'vae_loss': 1.384, 'gen_loss': 1.352}, val={'g_nll': 0.051, 'g_kl': 0.017, 'g_elbo': 0.067, 'x_rna_nll': 0.373, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.376, 'x_adt_nll': 0.859, 'x_adt_kl': 0.123, 'x_adt_elbo': 0.982, 'dsc_loss': 0.62, 'vae_loss': 1.36, 'gen_loss': 1.329}

2023-10-23 12:48:47,318 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


[INFO] EarlyStopping: Restoring checkpoint "49"...
[INFO] EarlyStopping: Restoring checkpoint "49"...
[INFO] fit_SCGLUE: Estimating balancing weight...
[INFO] estimate_balancing_weight: Clustering cells...
[INFO] estimate_balancing_weight: Matching clusters...
[INFO] estimate_balancing_weight: Matching array shape = (25, 22)...
[INFO] estimate_balancing_weight: Estimating balancing weight...


...

[INFO] fit_SCGLUE: Fine-tuning SCGLUE model...
[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] check_graph: All checks passed!
[INFO] SCGLUEModel: Setting `graph_batch_size` = 4868
[INFO] SCGLUEModel: Setting `align_burnin` = 18
[INFO] SCGLUEModel: Setting `max_epochs` = 105
[INFO] SCGLUEModel: Setting `patience` = 9
[INFO] SCGLUEModel: Setting `reduce_lr_patience` = 5
[INFO] SCGLUETrainer: Using training directory: "/tmp/GLUETMP_bxmjqw9"


/mnt/ps/home/CORP/johnny.xi/.pyenv/versions/3.10.12/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


[INFO] SCGLUETrainer: [Epoch 10] train={'g_nll': 0.026, 'g_kl': 0.018, 'g_elbo': 0.045, 'x_rna_nll': 0.374, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.376, 'x_adt_nll': 0.87, 'x_adt_kl': 0.127, 'x_adt_elbo': 0.997, 'dsc_loss': 0.646, 'vae_loss': 1.375, 'gen_loss': 1.342}, val={'g_nll': 0.026, 'g_kl': 0.018, 'g_elbo': 0.045, 'x_rna_nll': 0.372, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.375, 'x_adt_nll': 0.842, 'x_adt_kl': 0.126, 'x_adt_elbo': 0.968, 'dsc_loss': 0.676, 'vae_loss': 1.345, 'gen_loss': 1.311}, 9.0s elapsed
[INFO] SCGLUETrainer: [Epoch 20] train={'g_nll': 0.028, 'g_kl': 0.018, 'g_elbo': 0.046, 'x_rna_nll': 0.377, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.38, 'x_adt_nll': 0.87, 'x_adt_kl': 0.128, 'x_adt_elbo': 0.997, 'dsc_loss': 0.594, 'vae_loss': 1.379, 'gen_loss': 1.349}, val={'g_nll': 0.029, 'g_kl': 0.018, 'g_elbo': 0.046, 'x_rna_nll': 0.375, 'x_rna_kl': 0.003, 'x_rna_elbo': 0.378, 'x_adt_nll': 0.846, 'x_adt_kl': 0.127, 'x_adt_elbo': 0.972, 'dsc_loss': 0.639, 'vae_loss': 1.352, 'gen_loss': 1.32

2023-10-23 12:55:44,068 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


[INFO] EarlyStopping: Restoring checkpoint "47"...
[INFO] EarlyStopping: Restoring checkpoint "47"...


In [18]:
rna_encodings = glue.encode_data("rna", gex_ad)
adt_encodings = glue.encode_data("adt", adt_ad)

In [24]:
import os
import sys

In [67]:
sys.path.insert(0, os.path.abspath(".."))
from matching.utils import snn_matching, eot_matching, calc_domainAveraged_FOSCTTM

In [34]:
rna_encodings.shape

(16311, 50)

In [52]:
cell_types = np.unique(gex_ad.obs.cell_type)

In [56]:
import warnings
warnings.filterwarnings('ignore')

In [66]:
gex_ad.obsm["X_pca"].shape

(16311, 100)

In [84]:
knn_trace_avg, eot_trace_avg, knn_foscttm_avg, eot_foscttm_avg = 0, 0, 0, 0

for ct in cell_types:
    idx = np.where(gex_ad.obs.cell_type == ct)
    rna_sub, adt_sub = gex_ad.obsm["X_pca"][idx], adt_ad.obsm["X_pca"][idx]
    rna_match_sub, adt_match_sub = rna_encodings[idx], adt_encodings[idx]
    print(f"Cell type: {ct}, Number of samples: {rna_sub.shape[0]}")
    snn_sub = snn_matching(rna_match_sub, adt_match_sub)
    print(f"Cell type: {ct}, kNN trace: {np.trace(snn_sub)}")
    eot_sub = eot_matching(rna_match_sub, adt_match_sub, use_sinkhorn_log = True, verbose = False, max_iter = 200)
    print(f"Cell type: {ct}, EOT trace: {np.trace(eot_sub)}")
    snn_match = snn_sub @ adt_sub
    eot_match = eot_sub @ adt_sub
    knn_foscttm = np.array(calc_domainAveraged_FOSCTTM(adt_sub, snn_match)).mean()
    eot_foscttm = np.array(calc_domainAveraged_FOSCTTM(adt_sub, eot_match)).mean()
    print(f"Cell type: {ct}, kNN FOSCTTM: {knn_foscttm}") 
    print(f"Cell type: {ct}, EOT FOSCTTM: {eot_foscttm}") 

    knn_trace_avg += np.trace(snn_sub)/len(cell_types)
    eot_trace_avg += np.trace(eot_sub)/len(cell_types)
    knn_foscttm_avg += knn_foscttm/len(cell_types)
    eot_foscttm_avg += eot_foscttm/len(cell_types)

print(f"Average kNN Trace {knn_trace_avg}")
print(f"Average EOT Trace {eot_trace_avg}")
print(f"Average kNN FOSCTTM {knn_foscttm_avg}")
print(f"Average EOT FOSCTTM {eot_foscttm_avg}")


Cell type: B1 B IGKC+, Number of samples: 140
Cell type: B1 B IGKC+, kNN trace: 1.0227272727272727
Cell type: B1 B IGKC+, EOT trace: 1.8762991316152493
Cell type: B1 B IGKC+, kNN FOSCTTM: 0.4436793422404933
Cell type: B1 B IGKC+, EOT FOSCTTM: 0.47931654676258995
Cell type: B1 B IGKC-, Number of samples: 142
Cell type: B1 B IGKC-, kNN trace: 2.715909090909091
Cell type: B1 B IGKC-, EOT trace: 1.664947575325087
Cell type: B1 B IGKC-, kNN FOSCTTM: 0.43399760263709913
Cell type: B1 B IGKC-, EOT FOSCTTM: 0.4507042253521127
Cell type: CD14+ Mono, Number of samples: 1769
Cell type: CD14+ Mono, kNN trace: 1.0794285714285714
Cell type: CD14+ Mono, EOT trace: 2.037638130230977
Cell type: CD14+ Mono, kNN FOSCTTM: 0.4258766808458392
Cell type: CD14+ Mono, EOT FOSCTTM: 0.4745625388477781
Cell type: CD16+ Mono, Number of samples: 215
Cell type: CD16+ Mono, kNN trace: 0.02641569123695034
Cell type: CD16+ Mono, EOT trace: 0.7820789631029544
Cell type: CD16+ Mono, kNN FOSCTTM: 0.4317104977178874
Cell t

In [27]:
snn_coupling = snn_matching(rna_encodings, adt_encodings)


In [29]:
np.trace(snn_coupling)

2.0